In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define Image Size
IMG_SIZE = (256, 256)  # Updated image size

# Load Data
data_dir = "/content/drive/MyDrive/Converted Dataset"
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    subset='training'
)
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Build CNN Model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile Model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train Model
epochs = 20
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

# Evaluate model
y_true = val_generator.classes
y_pred = (model.predict(val_generator) > 0.5).astype(int).flatten()

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Specificity: {specificity:.4f}")
# Print Metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Specificity: {specificity:.4f}")


Mounted at /content/drive
Found 8741 images belonging to 2 classes.
Found 2184 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 2337s 8s/step - accuracy: 0.6503 - loss: 0.6783 - val_accuracy: 0.7674 - val_loss: 0.6060
Epoch 2/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 55s 199ms/step - accuracy: 0.8947 - loss: 0.2431 - val_accuracy: 0.7880 - val_loss: 0.6091
Epoch 3/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 55s 201ms/step - accuracy: 0.9415 - loss: 0.1338 - val_accuracy: 0.8077 - val_loss: 0.5496
Epoch 4/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 82s 201ms/step - accuracy: 0.9683 - loss: 0.0904 - val_accuracy: 0.8265 - val_loss: 0.6639
Epoch 5/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 56s 204ms/step - accuracy: 0.9871 - loss: 0.0379 - val_accuracy: 0.8370 - val_loss: 0.9304
Epoch 6/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 55s 199ms/step - accuracy: 0.9868 - loss: 0.0367 - val_accuracy: 0.8452 - val_loss: 0.7229
Epoch 7/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 55s 200ms/step - accuracy: 0.9906 - loss: 0.0320 - val_accuracy: 0.8274 - val_loss: 0.8749
Epoch 8/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 66s 240ms/step - accuracy: 0.9904 - loss: 0.

In [ ]:
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Save the model as 'heart1.pth'
torch.save(model.state_dict(), 'heart1.pth')
print("Model saved as 'heart1.pth'")

# Generate Confusion Matrix
def plot_confusion_matrix(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.title('Confusion Matrix')
    plt.show()
